### Q3. Graph Colouring Problem

b) Solving the primal and dual problems for both graphs

In [1]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

In [ ]:
#Defining both graphs - Vertices and Edges
V_1 = [1,2,3,4]
E_1 = [(1,2),(1,3),(1,4),(2,3),(2,4),(3,4)]
V_2 = [1,2,3,4,5]
E_2 = [(1,2),(2,3),(3,4),(4,5),(5,1)]

In [ ]:
#Converting the graphs into 0-indexed format for python
def convert_0_index(V,E):
    V_zero_index = [v - 1 for v in V]
    E_zero_index = [(u - 1,v - 1) for u,v in E]
    return V_zero_index,E_zero_index

In [ ]:
#Primal Optimisation problem solver
def primal_solver(V,E):
    #Converting to 0-index
    V_0, E_0 = convert_0_index(V,E)
    
    num_nodes = len(V_0)
    
    #Defining variables
    rho = cp.Variable()
    G = cp.Variable((num_nodes,num_nodes),symmetric=True)
    constraints = []
    
    #Constraints
    constraints.append(G >> 0)
    for v in V_0:
        constraints.append(G[v,v] == 1)
    
    for (u,v) in E_0:
        constraints.append(G[u,v] <= rho)
    
    objective = cp.Minimize(rho)
    problem = cp.Problem(objective, constraints)
    problem.solve()
    
    print("status:", problem.status)
    return problem.value, rho.value

In [ ]:
#Solving primal for graph 1
primal_obj_1, rho_star_1 = primal_solver(V_1,E_1)
print("For Graph 1:")
print("optimal value of primal objective:", primal_obj_1)
print("Primal optimiser rho*:", rho_star_1)

status: optimal
For Graph 1:
optimal value of primal objective: -0.333333332557487
Primal optimiser rho*: -0.333333332557487


- Graph 1:
    1. Optimal value of primal objective $p^*$: -0.333333332557487
    2. $\rho^*$ = -0.333333332557487

In [ ]:
#Solving primal for graph 2
primal_obj_2, rho_star_2 = primal_solver(V_2,E_2)
print("For Graph 2:")
print("optimal value of primal objective:", primal_obj_2)
print("Primal optimiser rho*:", rho_star_2)

status: optimal
For Graph 2:
optimal value of primal objective: -0.8090172334287293
Primal optimiser rho*: -0.8090172334287293


- Graph 2:
    1. Optimal value of primal objective $p^*$: -0.8090172334287293
    2. $\rho^*$ = -0.8090172334287293

In [ ]:
#Dual Optimisation problem solver (Minimisation form)
def dual_solver(V,E):
    V_0, E_0 = convert_0_index(V,E)
    num_nodes = len(V_0)
    
    constraints = []
    S = cp.Variable((num_nodes,num_nodes),symmetric=True)   #Dual Slack S
    lambda_vec = cp.Variable(num_nodes) #Lambda for G_{vv} = 1
    alpha = cp.Variable(len(E_0))   #alpha_{uv}
    
    #Constraints
    constraints.append(S >> 0)
    constraints.append(cp.sum(alpha) == 1)
    constraints.append(alpha >= 0)
    
    #Building matrix A
    A = 0
    for idx,(u,v) in enumerate(E_0):
        e = np.zeros((num_nodes,num_nodes))
        e[u,v] = 1
        e[v,u] = 1
        A += 0.5 * alpha[idx] * e
    
    S_expr = cp.diag(lambda_vec) + A
    constraints.append(S == S_expr)
    
    objective = cp.Minimize(cp.sum(lambda_vec))
    problem = cp.Problem(objective,constraints)
    
    problem.solve()
    
    print("status:", problem.status)
    dual_objective = - problem.value
    return dual_objective

In [ ]:
#Solving dual for graph 1
dual_obj_1 = dual_solver(V_1,E_1)
print("For Graph 1:")
print(f"Optimal value of dual objective: {dual_obj_1}")
print(f"Duality gap: {np.abs(dual_obj_1 - primal_obj_1)}")

status: optimal
For Graph 1:
Optimal value of dual objective: -0.3333374814223773
Duality gap: 4.148864890307635e-06


- Graph 1:
    1. Optimal value of dual objective $d^*$: -0.3333374814223773
    2. Duality gap $|d^* - p^*|$ = 4.148864890307635e-06

In [ ]:
#Solving dual for graph 2
dual_obj_2 = dual_solver(V_2,E_2)
print("For Graph 2:")
print(f"Optimal value of dual objective: {dual_obj_2}")
print(f"Duality gap: {np.abs(dual_obj_2 - primal_obj_2)}")

status: optimal
For Graph 2:
Optimal value of dual objective: -0.809016892936077
Duality gap: 3.404926522954099e-07


- Graph 2:
    1. Optimal value of dual objective $d^*$ = -0.809016892936077
    2. Duality gap $|d^* - p^*|$ = 3.404926522954099e-07